In [1]:
"""
 * @ Author: Lucas Glasner (lgvivanco96@gmail.com)
 * @ Create Time: 1969-12-31 21:00:00
 * @ Modified by: Lucas Glasner, 
 * @ Modified time: 2024-04-01 17:17:25
 * @ Description:
        Python translation of the "rational method" for computing peak runoff
        on drainage basins. 
 * @ Dependencies:
 */

"""
import pandas as pd
import numpy as np
import tqdm
import os

from src.rain import duration_coef
from src.infiltration import cn_correction
from src.geomorphology import tc_basin
from src.misc import to_numeric

In [2]:
# ------------------------------- main routines ------------------------------ #
def parse_params(params_path, **kwargs):
    """
    Load input data and preprocess for the model run

    Args:
        params_path (str): Path to textfile or spreadsheet
                               with basin parameters. 

    Returns:
        (tuple): formatted parameters,
                 daily rainfall
    """    
    file_ext = params_path.split('.')[-1]
    if (file_ext == 'csv') or (file_ext == 'txt'):
        params = pd.read_csv(params_path, index_col=0, **kwargs)
    elif (file_ext == 'xlsx') or (file_ext == 'xls'):
        params = pd.read_excel(params_path, index_col=0, **kwargs)
    else:
        raise ValueError('Parameter extension file must be ".xlsx" or ".csv"')
    
    params = params.map(lambda x: to_numeric(x))
    
    pr_mm24hr = params[params.index.map(lambda x: 'pr' in x)]
    return_periods = pr_mm24hr.index.map(
        lambda x: int(x.split('_')[1].replace('T', '')))
    pr_mm24hr.index = return_periods
    
    params.loc['curvenumber_1'] = [cn_correction(cn, amc)
                                        for cn, amc in zip(
                                            params.loc['curvenumber_1'],
                                            params.loc['moisturecondition_1'])]
    params = params.loc[['name','area_km2', 'mriverlen_km', 'meanslope_1',
                         'hmax_m', 'hmin_m', 'hmed_m', 'runoffcoef_1',
                         'curvenumber_1']]
    return params, pr_mm24hr
    

def RationalMethod(parameters, pr_mm24hr):
    """
    Given dataframes with basin parameters and precipitations this function
    computes the peak runoff of the rational method. 

    Args:
        parameters (DataFrame): pandas DataFrame with basin parameters
        pr_mm24hr (DataFrame): pandas DataFrame with precipitation for
            each basin and return period.

    Returns:
        basin_tcs, pr_mmXhr, peakrunoff (tuple):
            DataFrames with concentration time, precipitation intensity at the
            concentration time and model's peak runoff. 
    """
    # Runoff coefficient "frequency factors"
    runoff_FF  = pd.Series([1, 1, 1, 1.1, 1.15, 1.2,1.225, 1.25, 1.275, 1.3],
                        index=[2,5,10,20,25,50,75,100,150,200])
    pr_mm24hr = pr_mm24hr.copy()
    parameters     = parameters.copy()
    # Compute concentration time and change units to hours
    basin_tcs = tc_basin(parameters)/60
    basin_tcs['tc_mean_hr'] = basin_tcs[['tc_spain_hr', 'tc_kirpich_hr',
                                         'tc_SCS_hr', 'tc_california_hr']].mean(
                                             axis=1).values

    # Compute the duration coefficient for the concentration time
    DCoeffs = [duration_coef(basin_tcs.iloc[:,i].values)
                            for i in range(len(basin_tcs.columns))]
    DCoeffs = pd.DataFrame(DCoeffs,
                           columns=basin_tcs.index,
                           index=basin_tcs.columns)
    # Compute precipitation associated with the concentration time duration
    pr_mmXhr = pd.concat([pr_mm24hr*DCoeffs.loc[method]/basin_tcs[method]
                            for method in DCoeffs.index],
                            keys=DCoeffs.index)
    # Compute runoff coefficient for all basins and return periods
    runoff_coefs = [runoff_FF*parameters.loc['runoffcoef_1'].loc[basin]
                    for basin in parameters.columns]
    runoff_coefs = pd.concat(runoff_coefs, keys=parameters.columns).unstack().T

    # Compute peak runoff
    basin_areas  = parameters.loc['area_km2']
    peakrunoff   = [runoff_coefs*pr_mmXhr.loc[method]*basin_areas/3.6
                    for method in DCoeffs.index]
    peakrunoff   = pd.concat(peakrunoff, keys=DCoeffs.index)
    return basin_tcs.T, pr_mmXhr, peakrunoff

In [3]:
%%time
# ------------------------------ load input data ----------------------------- #
params, precips_mm24hr = parse_params('data/params.xlsx', sheet_name='racional')
precips_mm24hr = precips_mm24hr.T.dropna(how='all').T
params = params[precips_mm24hr.columns]
# ------------------------- run model for all basins ------------------------- #
basin_tcs, precips_mmXhr, peakrunoff = RationalMethod(params, precips_mm24hr)

# ------------------------ save results for each basin ----------------------- #
for fid in tqdm.tqdm(params.columns, total=len(params.columns)):
    name = params[fid].loc['name'].replace(' ','')
    try:
        os.mkdir(f'data/BasinsRacional/{name}')
        os.mkdir(f'data/BasinsRacional/{name}/tmp')
    except:
        pass
    
    out = f'data/BasinsRacional/{name}/QRational_{name}.xlsx'
    with pd.ExcelWriter(out, mode='w', engine='openpyxl') as writer:
        pd.concat([params[fid], basin_tcs[fid]]).to_excel(
            writer, sheet_name='concentration_time_hr')
        
    with pd.ExcelWriter(out, mode='a', engine='openpyxl') as writer:
        precips_mmXhr[fid].unstack().to_excel(
            writer, sheet_name='rainfall_intensity_mmhr-1')
        peakrunoff[fid].unstack().to_excel(
            writer, sheet_name='peakrunoff_m3s')
        
# --------------------------- save results summary --------------------------- #
out = f'data/BasinsRacional/QRational_Summary.xlsx'
with pd.ExcelWriter(out, mode='w', engine='openpyxl') as writer:
    params.to_excel(writer, sheet_name='parameters')
with pd.ExcelWriter(out, mode='a', engine='openpyxl') as writer:
    basin_tcs.to_excel(writer, sheet_name='concentration_time_hr')
    precips_mmXhr.to_excel(writer, sheet_name='rainfall_intensity_mmhr-1')
    peakrunoff.to_excel(writer, sheet_name='peakrunoff_m3s')

c:\lucas\P-projects\hydrocivilpy\src\rain.py:91: UserWarning: Storm duration is less than 0.16666666666666666 threshold, changing to 0.16666666666666666.
  warnings.warn(text)
100%|██████████| 26/26 [00:01<00:00, 20.89it/s]


CPU times: total: 141 ms
Wall time: 1.86 s
